In [1]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup

In [2]:
page = wptools.page('Sabra_and_Shatila_massacre')
page.get_parse()

en.wikipedia.org (parse) Sabra_and_Shatila_massacre
en.wikipedia.org (imageinfo) File:Bodies of victims of the massac...
Sabra and Shatila massacre (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Bodies of...
  infobox: <dict(11)> title, partof, image, caption, location, coo...
  pageid: 52268
  parsetree: <str(94600)> <root><template><title>Short description...
  requests: <list(2)> parse, imageinfo
  title: Sabra and Shatila massacre
  wikibase: Q208199
  wikidata_url: https://www.wikidata.org/wiki/Q208199
  wikitext: <str(76468)> {{Short description|1982 killing of civil...
}


In [3]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Sabra_and_Shatila_massacre",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Sabra_and_Shatila_massacre')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
1948 Palestinian exodus from Lydda and Ramle
1978 South Lebanon conflict
1982 Beirut bombing
1982 Hama massacre
1982 Iranian diplomats kidnapping
1982 Lebanon War
1983 Beirut barracks bombings
1983 United States embassy bombing
1984 Sohmor massacre
1984 United States embassy annex bombing in Beirut
1985 Beirut car bombings
Abu Nidal
Abu Shusha
Aharon Barak
Ahmad Tamal
Al-Dawayima massacre
Al-Manam
Al Bureij massacre
Alain Menargues
Aljazeera
Amal Hamdan
Amir Drori
Amnon Kapeliouk
Amos Yaron
Arab al-Mawasi massacre
Ariel Sharon
Assassination of Bachir Gemayel
Assassination of René Moawad
At Tiri incident
Avi Duda'i
Avraham Shalom
BBC
Bachir Gemayel
Balad al-Shaykh massacre
Barnaby Mason
Bashir Gemayel
Battle of Sidon (1991)
Battle of Tripoli (1983)
Battle of Zahleh
Battle of the Hotels
Bayt Daras
Beirut
Benny Morris
Bernard Lewis
Black Saturday (Lebanon)
Bombing of Lebanon (June 1981)
Brandeis University
Bus massacre
Castration
Cave of the Patriarchs massacre
Chris Tolworthy
Chri

In [4]:
def get_arabic_name(english_phrase):
    # if we remove all numbers from a string and the string remains empty - then we won't translate that
    english_phrase_mod = ''.join([i for i in english_phrase if not i.isdigit()])
    english_phrase_mod = english_phrase_mod.translate(str.maketrans('', '', string.punctuation))
    english_phrase_mod = english_phrase_mod.strip()
    if english_phrase_mod == '':
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1
    else:
        english_phrase = english_phrase_mod
    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries
    
    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s

def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [5]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [6]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [7]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1
                    
                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    print(name)
                    if isinstance(name, list):
                        if len(name) > 1:
                            print(name)
                            jsims = []
                            for i, n in enumerate(name):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name = [name[jsims.index(max(jsims))]]
                            print('name after: ', name)
            
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'
                    
                    df_political_party_en = df_political_party_en.append({
                        'Name': ','.join(name if isinstance(name, list) else name),
                        'Ideology': ','.join(ideology) if isinstance(ideology, list) else ideology,
                        'Political Position': ','.join(position) if isinstance(position, list) else position,
                        'Founder(s)': founders,
                    }, ignore_index=True)
                    
                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)
                    
                    
                    
                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']
                    
                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'
                    
                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')
                    
                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'
                    
                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')
                    
                    
                    df_political_party_ar = df_political_party_ar.append({
                        'الاسم': ','.join(name_ar),
                        'الإيديولوجيا': ','.join(ideology_ar),
                        'الموقف السياسي': ','.join(position_ar),
                        'المؤسس (المؤسسون)': ','.join(founders_ar),
                    }, ignore_index=True)
                    
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1948 Palestinian exodus from Lydda and Ramle
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:PikiWiki Israel 4220 Israel Def...
en.wikipedia.org (parse) 1982 Beirut bombing
en.wikipedia.org (parse) 1982 Hama massacre
en.wikipedia.org (imageinfo) File:1982HamaMassacre.png
en.wikipedia.org (parse) 1982 Iranian diplomats kidnapping
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1983 Beirut barracks bombings
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 United States embassy bombing
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
en.wikipedia.org (parse) 1984 Sohmor massacre
en.wikipedia.org (parse) 1984 United States embassy annex bombing...
en.wikipedia.org (imageinfo) File:US Embassy in Beirut, Lebanon (...
en.wikipedia.org (parse) 1985 Beirut car bombings
en.wikipedia.org (imageinfo) File:Bir al-Abed massacre.PNG
en.wikipedia.org (parse) Abu Nidal
en.wikipedia.org (i

en.wikipedia.org (parse) John Harbo
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Kafr Qasim massacre
en.wikipedia.org (imageinfo) File:KafrQasimMemorial.jpg
en.wikipedia.org (parse) Kahan Commission
en.wikipedia.org (parse) Kahan commission
en.wikipedia.org (parse) Karantina massacre
en.wikipedia.org (imageinfo) File:Karantina Massacre.jpg
en.wikipedia.org (parse) Khan Yunis massacre
en.wikipedia.org (imageinfo) File:Khan Yunis-1930s.jpg
en.wikipedia.org (parse) Khobar Towers bombing
en.wikipedia.org (imageinfo) File:AnschalgInZahran1996 KhobarTower.jpg
en.wikipedia.org (parse) Kurds in Lebanon
en.wikipedia.org (parse) Le Monde Diplomatique
en.wikipedia.o

[[Samir Geagea]]
[[Bachir Gemayel]]
-----------------
name:  ['Lebanese Forces']
founders:  Samir Geagea,Bachir Gemayel,
ideology:  ['Liberal conservatism', 'Christian nationalism', 'Lebanese nationalism', 'National conservatism']
position:  ['Right-wing']
START
name:  Lebanese Forces
name ar:  القوات اللبنانية 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%82%D9%88%D8%A7%D8%AA_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
START
ideology:  Liberal conservatism
ideology ar:  محافظة ليبرالية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D8%A7%D9%81%D8%B8%D8%A9_%D9%84%D9%8A%D8%A8%D8%B1%D8%A7%D9%84%D9%8A%D8%A9
ideology:  Christian nationalism
ideology ar:  قومية مسيحية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%85%D8%B3%D9%8A%D8%AD%D9%8A%D8%A9
ideology:  Lebanese nationalism
ideology ar:  قومية لبنانية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%

en.wikipedia.org (parse) Lebanese Forces (militia)
en.wikipedia.org (imageinfo) File:Forces Libanaises Flag.svg


name before:  القوات اللبنانية
name after:  ['القوات اللبنانية']
-----------------
name:  ['القوات اللبنانية']
founders:  
ideology:  ['Conservatism', 'Christian nationalism']
position:  -
START
Cannot translate phrase `القوات اللبنانية` to arabic
END
START
ideology:  Conservatism
ideology ar:  سياسة محافظة 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B3%D9%8A%D8%A7%D8%B3%D8%A9_%D9%85%D8%AD%D8%A7%D9%81%D8%B8%D8%A9
ideology:  Christian nationalism
ideology ar:  قومية مسيحية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%85%D8%B3%D9%8A%D8%AD%D9%8A%D8%A9
END
START
END


...

en.wikipedia.org (parse) Syrian intervention in the Lebanese Civil War
en.wikipedia.org (parse) Syrians in Lebanon
en.wikipedia.org (parse) Tantura
en.wikipedia.org (parse) Tel al-Zaatar massacre
en.wikipedia.org (imageinfo) File:JeanJacquesKurz-DestructionOfTe...
en.wikipedia.org (parse) The Daily Telegraph
en.wikipedia.org (imageinfo) File:The Daily Telegraph (British ne...
en.wikipedia.org (parse) The Independent
en.wikipedia.org (imageinfo) File:The Independent screenshot, 25 ...
en.wikipedia.org (parse) The Nation (U.S. periodical)
en.wikipedia.org (parse) The New York Times
en.wikipedia.org (imageinfo) File:The-New-York-Times-March-26-2018.jpg
en.wikipedia.org (parse) Thomas Friedman
en.wikipedia.org (imageinfo) File:Thomas Friedman 2005 (5).jpg
en.wikipedia.org (parse) Thomas L. Friedman
en.wikipedia.org (imageinfo) File:Thomas Friedman 2005 (5).jpg
en.wikipedia.org (parse) Time (magazine)
en.wikipedia.org (parse) United Nations General Assembly
en.wikipedia.org (imageinfo) File

skipping Wikipedia:Protection policy


en.wikipedia.org (parse) File:Lebanon Beirut OSM transport.png
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Module:Location map/data/Lebanon Beirut


skipping Template:Campaignbox Lebanese Civil War
skipping Template:Massacres against Palestinians
skipping Template talk:Campaignbox Lebanese Civil War
skipping Template talk:Massacres against Palestinians
skipping Help:Authority control
skipping Category:Articles with GND identifiers
skipping Category:Articles with LCCN identifiers
skipping Category:Use dmy dates from September 2018


API error: {'code': 'notwikitext', 'info': '"prop=parsetree" is only supported for wikitext content.', 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}


In [8]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,Lebanese Forces,"Liberal conservatism,Christian nationalism,Leb...",Right-wing,"Samir Geagea,Bachir Gemayel,"
1,القوات اللبنانية,"Conservatism,Christian nationalism",-,
2,Palestine Liberation Organization,Palestinian nationalism,-,"Mahmoud Abbas,"
3,Palestine Liberation Organization,Palestinian nationalism,-,"Mahmoud Abbas,"
4,Syrian Social Nationalist Party,"Secularism,Syrian nationalism,officially denie...","Left-wing collapsible list,Syncretic","Fares al-Saad,Antoun Saadeh,"


In [9]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,القوات اللبنانية,"محافظة ليبرالية ,قومية مسيحية ,قومية لبنانية ,...",يمينية,"سمير جعجع ,بشير الجميل"
1,,"سياسة محافظة ,قومية مسيحية",-,
2,منظمة التحرير الفلسطينية,قومية فلسطينية,-,
3,منظمة التحرير الفلسطينية,قومية فلسطينية,-,
4,الحزب السوري القومي الاجتماعي,"علمانية ,القومية السورية ,رفض رسميًا) قائمة قا...","قائمة يسارية قابلة للطي (مترجمه),توفيق بين الأ...",


In [10]:
datasets_dir = 'datasets_updated/1982-1984/Sabra_and_Shatila_massacre/'

In [11]:
mkdir(datasets_dir)
df_political_party_en.to_csv(os.path.join(datasets_dir, 'political_parties_en.csv'), index=False)
df_political_party_ar.to_csv(os.path.join(datasets_dir, 'political_parties_ar.csv'), index=False, encoding='utf-8-sig')

In [12]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [13]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'
                
                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'
                    
                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                    
                    df_politician_en = df_politician_en.append({
                        'Name': name_inserted,
                        'Political Party': ','.join(party) if isinstance(party, list) else party,
                        'Nationality': ','.join(nationality) if isinstance(nationality, list) else nationality,
                        'Religion': ','.join(religion) if isinstance(religion, list) else religion,
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)
                    
                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)
                    
                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'
                    
                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'
                        
                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'
                    
                    df_politician_ar = df_politician_ar.append({
                        'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                        'الحزب السياسي': ','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar,
                        'الجنسيه': ','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar,
                        'الدين': ','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar,
                    }, ignore_index=True)
                    
                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)

        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1948 Palestinian exodus from Lydda and Ramle
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:PikiWiki Israel 4220 Israel Def...
en.wikipedia.org (parse) 1982 Beirut bombing
en.wikipedia.org (parse) 1982 Hama massacre
en.wikipedia.org (imageinfo) File:1982HamaMassacre.png
en.wikipedia.org (parse) 1982 Iranian diplomats kidnapping
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1983 Beirut barracks bombings
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 United States embassy bombing
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
en.wikipedia.org (parse) 1984 Sohmor massacre
en.wikipedia.org (parse) 1984 United States embassy annex bombing...
en.wikipedia.org (imageinfo) File:US Embassy in Beirut, Lebanon (...
en.wikipedia.org (parse) 1985 Beirut car bombings
en.wikipedia.org (imageinfo) File:Bir al-Abed massacre.PNG
en.wikipedia.org (parse) Abu Nidal
en.wikipedia.org (i

-----------------
name:  ['أبو نضال', 'Abu Nidal', 'ar', 'lang']
native_name:  -
nickname:  -
religion:  -
party:  -
nationality:  ['Palestinian']
Cannot translate phrase `Abu Nidal` to arabic
name ar:  -1
START
nationality:  Palestinian
nationality ar:  فلسطينيون 
nationality ar link:  https://ar.wikipedia.org/wiki/%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D9%88%D9%86
END
name ar:  -1
religion ar:  -
party ar:  -
nationality ar:  ['فلسطينيون ']


en.wikipedia.org (parse) Abu Shusha
en.wikipedia.org (parse) Aharon Barak
en.wikipedia.org (imageinfo) File:Aharon Barak.jpg


-----------------
name:  ['Aharon Barak']
native_name:  -
nickname:  -
religion:  -
party:  -
nationality:  ['Israeli']
name ar:  أهارون باراك 
START
nationality:  Israeli
nationality ar:  إسرائيلي (مترجمه)
nationality ar link:  -1
END
name ar:  أهارون باراك 
religion ar:  -
party ar:  -
nationality ar:  ['إسرائيلي (مترجمه)']


en.wikipedia.org (parse) Ahmad Tamal
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Al-Dawayima massacre
en.wikipedia.org (parse) Al-Manam
en.wikipedia.org (parse) Al Bureij massacre
en.wikipedia.org (imageinfo) File:IDF Caterpillar D9 bulldozer wo...
en.wikipedia.org (parse) Alain Menargues
en.wikipedia.org (parse) Aljazeera
en.wikipedia.org (parse) Amal Hamdan
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt

...

en.wikipedia.org (parse) Young men (Lebanon)
en.wikipedia.org (parse) Ze'ev Schiff
en.wikipedia.org (parse) Zeitoun incident
en.wikipedia.org (imageinfo) File:Samouni2.jpg


skipping Wikipedia:Protection policy


en.wikipedia.org (parse) File:Lebanon Beirut OSM transport.png
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Module:Location map/data/Lebanon Beirut


skipping Template:Campaignbox Lebanese Civil War
skipping Template:Massacres against Palestinians
skipping Template talk:Campaignbox Lebanese Civil War
skipping Template talk:Massacres against Palestinians
skipping Help:Authority control
skipping Category:Articles with GND identifiers
skipping Category:Articles with LCCN identifiers
skipping Category:Use dmy dates from September 2018


API error: {'code': 'notwikitext', 'info': '"prop=parsetree" is only supported for wikitext content.', 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}


In [14]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,"(أبو نضال,Abu Nidal,ar,lang,)",-,Palestinian,-
1,"(Aharon Barak,)",-,Israeli,-
2,"(Amir Drori,)",-,-,-
3,"(Amos Yaron,)",-,-,-
4,"(-,)",plainlist,-,-
5,"(ar,lang,Bachir Gemayel,بشير الجميّل,)",Kataeb Party,-,-
6,"(ar,رينيه معوض,René Moawad<Br/> Nobold,lang,ري...",Independent,-,-
7,"(ar,lang,Bachir Gemayel,بشير الجميّل,)",Kataeb Party,-,-
8,"(ar,lang,Bachir Gemayel,بشير الجميّل,)",Kataeb Party,-,-
9,"(Benny Morris,)",-,Israeli,-


In [15]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,فلسطينيون,-,-1
1,-,إسرائيلي (مترجمه),-,أهارون باراك
2,-,-,-,أمير دروري (مترجمه)
3,-,-,-,عاموس يارون (مترجمه)
4,-,-,قائمة عادية (مترجمه),أرئيل شارون
5,-,-,حزب الكتائب اللبنانية,بشير الجميل
6,-,-,مستقل (توضيح),رينيه معوض
7,-,-,حزب الكتائب اللبنانية,بشير الجميل
8,-,-,حزب الكتائب اللبنانية,بشير الجميل
9,-,إسرائيلي (مترجمه),-,بيني موريس


In [16]:
mkdir(datasets_dir)
df_politician_en.to_csv(os.path.join(datasets_dir, 'politicians_en.csv'), index=False)
df_politician_ar.to_csv(os.path.join(datasets_dir, 'politicians_ar.csv'), index=False, encoding='utf-8-sig')

In [17]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [18]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)
                    
                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'
    
                    df_locations_en = df_locations_en.append({
                        'Name': ','.join(name) if isinstance(name, list) and len(name) > 1 else name,
                        'Subdivision type': ','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type,
                        'Subdivision name': ','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name                    
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)
                    
                    
                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass
                    
                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'
        

                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = df_locations_ar.append({
                            'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                            'نوع التقسيم': ','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar,
                            'اسم التقسيم': ','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar,
                        }, ignore_index=True)
                    
                    else:
                        print('skipping {} because name is not found'.format(p))
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1948 Palestinian exodus from Lydda and Ramle
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:PikiWiki Israel 4220 Israel Def...
en.wikipedia.org (parse) 1982 Beirut bombing
en.wikipedia.org (parse) 1982 Hama massacre
en.wikipedia.org (imageinfo) File:1982HamaMassacre.png
en.wikipedia.org (parse) 1982 Iranian diplomats kidnapping
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1983 Beirut barracks bombings
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 United States embassy bombing
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
en.wikipedia.org (parse) 1984 Sohmor massacre
en.wikipedia.org (parse) 1984 United States embassy annex bombing...
en.wikipedia.org (imageinfo) File:US Embassy in Beirut, Lebanon (...
en.wikipedia.org (parse) 1985 Beirut car bombings
en.wikipedia.org (imageinfo) File:Bir al-Abed massacre.PNG
en.wikipedia.org (parse) Abu Nidal
en.wikipedia.org (i

-----------------
name:  Abu Shusha
subdivision_type:  ['Geopolitical entity']
subdivision_name:  ['Mandatory Palestine']
name ar:  أبو شوشة (الرملة) 
START
subdivision_type:  Geopolitical entity
subdivision_type ar:  علم الوجود الجغرافي السياسي 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_%D8%A7%D9%84%D9%88%D8%AC%D9%88%D8%AF_%D8%A7%D9%84%D8%AC%D8%BA%D8%B1%D8%A7%D9%81%D9%8A_%D8%A7%D9%84%D8%B3%D9%8A%D8%A7%D8%B3%D9%8A
END
START
subdivision_name:  Mandatory Palestine
subdivision_name ar:  الانتداب البريطاني على فلسطين 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%A7%D9%86%D8%AA%D8%AF%D8%A7%D8%A8_%D8%A7%D9%84%D8%A8%D8%B1%D9%8A%D8%B7%D8%A7%D9%86%D9%8A_%D8%B9%D9%84%D9%89_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86
END


en.wikipedia.org (parse) Aharon Barak
en.wikipedia.org (imageinfo) File:Aharon Barak.jpg
en.wikipedia.org (parse) Ahmad Tamal
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Al-Dawayima massacre
en.wikipedia.org (parse) Al-Manam
en.wikipedia.org (parse) Al Bureij massacre
en.wikipedia.org (imageinfo) File:IDF Caterpillar D9 bulldozer wo...
en.wikipedia.org (parse) Alain Menargues
en.wikipedia.org (parse) Aljazeera
en.wikipedia.org (parse) Amal Hamdan
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;h

-----------------
name:  Bayt Daras
subdivision_type:  ['Geopolitical entity']
subdivision_name:  ['Mandatory Palestine']
Cannot translate phrase `Bayt Daras` to arabic
name ar:  -1
START
subdivision_type:  Geopolitical entity
subdivision_type ar:  علم الوجود الجغرافي السياسي 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_%D8%A7%D9%84%D9%88%D8%AC%D9%88%D8%AF_%D8%A7%D9%84%D8%AC%D8%BA%D8%B1%D8%A7%D9%81%D9%8A_%D8%A7%D9%84%D8%B3%D9%8A%D8%A7%D8%B3%D9%8A
END
START
subdivision_name:  Mandatory Palestine
subdivision_name ar:  الانتداب البريطاني على فلسطين 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%A7%D9%86%D8%AA%D8%AF%D8%A7%D8%A8_%D8%A7%D9%84%D8%A8%D8%B1%D9%8A%D8%B7%D8%A7%D9%86%D9%8A_%D8%B9%D9%84%D9%89_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86
END


en.wikipedia.org (parse) Beirut


...

en.wikipedia.org (parse) William B. Quandt
en.wikipedia.org (imageinfo) File:William B. Quandt.jpg
en.wikipedia.org (parse) William Schabas
en.wikipedia.org (imageinfo) File:ICTY Legacy Symposium Panel III...
en.wikipedia.org (parse) World Press Review
en.wikipedia.org (parse) Yehoshua Saguy
en.wikipedia.org (imageinfo) File:Yehoshua Sagi.jpg
en.wikipedia.org (parse) Yitzhak Kahan
en.wikipedia.org (parse) Yitzhak Shamir
en.wikipedia.org (imageinfo) File:Yitzhak Shamir 1992 Dan Hadani ...
en.wikipedia.org (parse) Yona Efrat
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Yossi Alpher
en.wikipedia.org (imageinfo) File:YossiAlpher.jpg
en.wikipedia.org (parse) Y

skipping Wikipedia:Protection policy


en.wikipedia.org (parse) File:Lebanon Beirut OSM transport.png
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Module:Location map/data/Lebanon Beirut


skipping Template:Campaignbox Lebanese Civil War
skipping Template:Massacres against Palestinians
skipping Template talk:Campaignbox Lebanese Civil War
skipping Template talk:Massacres against Palestinians
skipping Help:Authority control
skipping Category:Articles with GND identifiers
skipping Category:Articles with LCCN identifiers
skipping Category:Use dmy dates from September 2018


API error: {'code': 'notwikitext', 'info': '"prop=parsetree" is only supported for wikitext content.', 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}


In [19]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,Abu Shusha,Geopolitical entity,Mandatory Palestine
1,Bayt Daras,Geopolitical entity,Mandatory Palestine
2,[Beirut],Country,Lebanon
3,[Iraq],-,-
4,[Israel],-,-
5,[Lebanon],-,-
6,Nahariyya,Country,ISR
7,Saliha,Geopolitical entity,Mandatory Palestine
8,[Singapore],-,-
9,Tantura,Geopolitical entity,Mandatory Palestine


In [20]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,الانتداب البريطاني على فلسطين,علم الوجود الجغرافي السياسي,أبو شوشة (الرملة)
1,الانتداب البريطاني على فلسطين,علم الوجود الجغرافي السياسي,-1
2,لبنان,بلد,بيروت
3,-,-,العراق
4,-,-,إسرائيل
5,-,-,لبنان
6,,بلد,نهاريا
7,الانتداب البريطاني على فلسطين,علم الوجود الجغرافي السياسي,صلحا
8,-,-,سنغافورة
9,الانتداب البريطاني على فلسطين,علم الوجود الجغرافي السياسي,الطنطورة (حيفا)


In [21]:
mkdir(datasets_dir)
df_locations_en.to_csv(os.path.join(datasets_dir, 'locations_en.csv'), index=False)
df_locations_ar.to_csv(os.path.join(datasets_dir, 'locations_ar.csv'), index=False, encoding='utf-8-sig')